### **Import Required Libraries**

In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.metrics import accuracy_score,precision_score,recall_score,classification_report
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline

### **Import Datasets**

In [ ]:
train_data=pd.read_csv("../dataset/train_data.csv")
test_data=pd.read_csv("../dataset/test_data.csv")

In [ ]:
print(f"Shape of the Training data:{train_data.shape}")
print(f"Shape of the Testing data:{test_data.shape}")

### **Drop unecessary Columns**

In [ ]:
train_data=train_data.drop(["Unnamed: 0"],axis=1)
test_data=test_data.drop(["Unnamed: 0"],axis=1)

In [ ]:
##split train data into the X_train and Y_train
X_train=train_data.drop(["delivery_status"],axis=1)
Y_train=train_data["delivery_status"]

### **Model Building**

In [ ]:
# ### Numbercal columns 
numerical_features=X_train.select_dtypes(include="number").columns.to_list()
categorical_features=X_train.select_dtypes(include="object").columns.to_list()

# ##Nominal variables
categorical_features.remove("brand")
##Ordinal variables
ordinal_features=["brand"]

In [ ]:
##mapping y_train variable with the numeric values
y_map={"Failed Delivery":0,"On Going":1,"Delivered":2}
Y_train=Y_train.map(y_map)

In [ ]:
### Define pipelines
nominal_pipeline=Pipeline(steps=[
    ("One-Hot-Encoder",OneHotEncoder(sparse_output=False,handle_unknown="ignore"))
])
ordinal_pipeline=Pipeline(steps=[
    ("Ordinal-Encoder",OrdinalEncoder())
])

transfomer=ColumnTransformer(transformers=[
    ("Nominal Pipeline",nominal_pipeline,categorical_features),
    ("Ordinal Pipeline",ordinal_pipeline,ordinal_features)
])

final_pipeline=Pipeline(steps=[
    ("Transformer",transfomer),
    ("Resampling SMOTE",SMOTE(random_state=42)), ##resample
    ("xgb",XGBClassifier(random_state=42))
])

## Execute Pipeline
final_pipeline.fit(X_train,Y_train)

### **Model Evaluation**

In [ ]:
##split test data into the X_test and Y_test
X_test=test_data.drop(["delivery_status"],axis=1)
Y_test=test_data["delivery_status"]

In [ ]:
##mapping y_test variable with the numeric values
y_map={"Failed Delivery":0,"On Going":1,"Delivered":2}
Y_test=Y_test.map(y_map)

In [ ]:
y_pred=final_pipeline.predict(X_test) ##get the predicted values

In [ ]:
##print the classification report
print(classification_report(Y_test,y_pred))

In [ ]:
y_pred_train=final_pipeline.predict(X_train)

In [ ]:
##print the classification report
print(classification_report(Y_train,y_pred_train))